In [19]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

emoji_examples = [
    ("human", "탑건"),
    ("ai",    "🛩️👨‍✈️🔥"),
    ("human", "대부"),
    ("ai",    "👨‍👨‍👦🔫🍝"),
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 제목을 입력받으면 **반드시 3개의 관련 이모티콘**만 출력하는 AI입니다.\n"
        ),
        MessagesPlaceholder(variable_name="history"), 
        *emoji_examples,                          
        ("human", "{question}")                     
    ]
)



def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [20]:
invoke_chain("인사이드 아웃")   

content='😊🧠💥'


In [21]:
invoke_chain("기생충")

content='🐜🏠💰'
